In [ ]:
import pytest
from unittest.mock import patch, MagicMock
from rss_collector.feed_parser import retry_with_backoff

def test_retry_with_backoff_success():
    """
    Test that retry_with_backoff succeeds after a few failed attempts.
    """
    test_url = "https://example.com/rss"

    # Mock `fetch_feed_content_with_proxy` to fail twice before succeeding
    with patch("rss_collector.feed_parser.fetch_feed_content_with_proxy") as mock_fetch:
        # First two attempts fail, third succeeds
        mock_fetch.side_effect = [None, None, "<rss>Valid Feed</rss>"]

        # Mock feedparser.parse to return a valid feed
        with patch("rss_collector.feed_parser.feedparser.parse") as mock_parse:
            mock_feed = MagicMock()
            mock_feed.feed = {"title": "Test Feed"}
            mock_feed.entries = []
            mock_parse.return_value = mock_feed

            # Call the function
            parsed_feed = retry_with_backoff(test_url, retries=3)

            # Debugging outputs
            print("Mock Fetch Call Count:", mock_fetch.call_count)
            print("Parsed Feed:", parsed_feed)

            # Assertions
            assert mock_fetch.call_count == 3, "Expected 3 retries, got: {mock_fetch.call_count}"
            assert parsed_feed is not None, "Expected parsed_feed to be not None."
            assert parsed_feed.feed["title"] == "Test Feed", f"Expected 'Test Feed', got: {parsed_feed.feed['title']}"

def test_retry_with_backoff_failure():
    """
    Test that retry_with_backoff returns None after all retries fail.
    """
    test_url = "https://example.com/rss"

    # Mock `fetch_feed_content_with_proxy` to always fail
    with patch("rss_collector.feed_parser.fetch_feed_content_with_proxy") as mock_fetch:
        mock_fetch.return_value = None

        # Call the function
        parsed_feed = retry_with_backoff(test_url, retries=3)

        # Assertions
        assert mock_fetch.call_count == 3, "Expected 3 retries, got: {mock_fetch.call_count}"
        assert parsed_feed is None, "Expected parsed_feed to be None."

